In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz, kelvin
from astropy.cosmology import Planck18 as cosmo
from scipy import signal

from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.filters import UVJ
from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.sed import Sed
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds
from synthesizer.kernel_functions import Kernel
from synthesizer.conversions import lnu_to_absolute_mag
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Blackbody, Greybody
from synthesizer.emission_models import (
    EmissionModel,
    AttenuatedEmission,
    BimodalPacmanEmission,
    DustEmission,
    EmissionModel,
    UnifiedAGN,
    CharlotFall2000,
    IncidentEmission,
    NebularEmission,
    ReprocessedEmission,
    StellarEmissionModel,
)
from synthesizer.conversions import lnu_to_absolute_mag, fnu_to_apparent_mag, fnu_to_lnu

In [2]:
# define functions

# Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
# In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.
# only need this apparent mag function if using Mats code for calculating UVLF.
def apparent_magnitude_from_absolute(redshift, absolute_magnitude):
    # Get the luminosity distance for the given redshift
    # Distnaces are in parsecs
    d_L_pc = Planck18.luminosity_distance(redshift).to('pc').value
    
    # Calculate the apparent magnitude using the distance modulus formula
    apparent_mag = absolute_magnitude - 5 + (5 * np.log10(d_L_pc))
    
    return apparent_mag

# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_df(ab_mag, volume, massBinLimits):

# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])

        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors

        return phi, phi_sigma, hist


In [3]:
# get grid for all galaxies
# gives grid of metalicity and age which is used to map on our camels galaxies
grid_name = "bc03_chabrier03-0.1,100.hdf5"
#grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5" # try old grid, LF looks weird?!
grid_dir = "/home/jovyan/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

In [4]:
# get gals
output_dir = '/home/jovyan/camels/CV_set/CV_outputs/LFs/'
directories = [f'CV_{i}' for i in range(0, 27)]
skipped_directories = []


In [5]:
# get filters:
# Need to use the grid lam, dont need to define wavelengths here: see codes here: http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Swift/UVOT.UVM2&&mode=browse&gname=Swift&gname2=UVOT#filter
# need to use transmission NOT area
# Get for XMM
fil_uvm2_XMM = Filter("XMM/OM.UVM2_filter", new_lam=grid.lam)

# now get for UVOT
fil_uvm2_UVOT = Filter("Swift/UVOT.UVM2_fil", new_lam=grid.lam) # changed from new_lam=lams_uvot to grid.lam and get 0 transmission, but warning sign is back

# what is a top hat filter?
filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)

# 0 = TopHat, 1 = XMM, 2= Swift
filt_lst = [filt1, fil_uvm2_XMM, fil_uvm2_UVOT]

# get filters in combined array to apply to all galaxies
combined_filters = FilterCollection(
    filters=filt_lst, new_lam=grid.lam
)


In [6]:
# set incident emission model
incident = IncidentEmission(grid)
# set little h value
little_h =  0.6711


In [7]:
for CV_X in directories:
    try:
        # create an empty dictionary to store your data - reset in loop
        data_dict = {}
        
        # Get grid for all galaxies
        dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/CV/' + CV_X
        gals_074 = load_CAMELS_IllustrisTNG(
            dir_,
            snap_name='snapshot_074.hdf5', 
            group_name='groups_074.hdf5',
        )
        
        cat_074 = dir_+'/groups_074.hdf5'
        
        # open file
        f = h5py.File(cat_074, 'r')
        
        # read different attributes of the header
        boxSize_074 = f['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
        redshift_074 = f['Header'].attrs[u'Redshift']

        # Filter galaxies to only include those with 100 or more star particles
        ## NEED TO ADD FILTER TO GET RID OF ANY GALAXIES WITH LESS THAN 100 star Particles!! This is the resolution limit!!
        # Chris: when you load your galaxies in could you do a filter for those with at least 100 star particles? that's the resolution limit
        
        # Filter galaxies to only include those with 100 or more star particles
        gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]
        gals_074 = [gal for gal in gals_074 if gal.gas.nparticles >= 100]
        #gals_074 = [gal for gal in gals_074 if len(gal.stars.ages) >= 100]  # this shouldnt be here?! donesnt make sense

        spec_list = []
        # Lets work with z=0 so gals_025
        for i in gals_074:
            gal = i
            # get_spectra_incident An Sed object containing the stellar spectra
            spec = gal.stars.get_spectra(incident)
            spec.get_fnu0()
            spec_list.append(spec)

        # combine our seds
        seds = combine_list_of_seds(spec_list)
        seds.lnu  # get luminosity

        # get rest frame photometry for our filters
        phot = seds.get_photo_lnu(combined_filters, verbose=True) 
        
        # now using the filters which got the rest frame lum, get the abs magnitude in those filters
        #abs_mags = [lnu_to_absolute_mag(phot[f]) for f in combined_filters.filters]
        # just want top hat filter for now
        abs_mag_th = [lnu_to_absolute_mag(phot[f]) for f in combined_filters.filters]

        # next steps, get luminosity function for these magnitudes
        # co-moving volume: BoxSize_025 and redshift:
        little_h =  0.6711
        Vphys = (boxSize_074/little_h )**3
        Vcom = Vphys * ((1+redshift_074)**3)
        
         # Define the bin edges for AB magnitudes
        # Here, let's define bins from -25 to -17 in steps of 0.5 magnitude
        massBinLimits = np.arange(-22, -16, 0.5)

        # calc LF
        phi, phi_sigma, hist = calc_df(abs_mag_th, Vcom, massBinLimits)
        
        # Define output file path
        output_file = f"{output_dir}{CV_X}.txt"

        # Write the data to the text file
        with open(output_file, 'w') as txtfile:
            # Write phi values
            txtfile.write("phi\n")
            for value in phi:
                txtfile.write(f"{value}\n")

            # Write phi_sigma values
            txtfile.write("phi_sigma\n")
            for value in phi_sigma:
                txtfile.write(f"{value}\n")

            # Write hist values
            txtfile.write("hist\n")
            for value in hist:
                txtfile.write(f"{value}\n")

            # Write massBinLimits values
            txtfile.write("massBinLimits\n")
            for value in massBinLimits:
                txtfile.write(f"{value}\n")

        print(f'Written out: {output_file}')

    except Exception as e:
        print(f"Error processing {CV_X}: {e}")
        skipped_directories.append(CV_X)

# Print skipped directories at the end
if skipped_directories:
    print("Skipped directories:", skipped_directories)

/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1949: RuntimeWarning: invalid value encountered in divide
  out_arr = func(


Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_0.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_1.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_2.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_3.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_4.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_5.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_6.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_7.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_8.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_9.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_10.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_11.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_12.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_13.txt
Written out: /home/jovyan/camels/CV_set/CV_outputs/LFs/CV_14.txt
Written out: /home/jovyan/camels/CV

# plot as test. will only do last one in array
label_025 = 'z = ', redshift_074
label_th = 'Tophat'



# Plot the luminosity function
plt.errorbar(massBinLimits[:-1], phi_th, yerr=phi_sigma_th, fmt='o', color='blue', label=label_th)

plt.xlabel('Absolute Magnitude (AB)')
plt.ylabel('Number Density (Mpc^-3 mag^-1)')
plt.yscale('log')
plt.legend(loc='lower right')


plt.title('Luminosity Function Top Hat filter')
plt.grid(True)
plt.show()